# **1. Importation du plugin pandas**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os

file_path_patoche = "/content/drive/MyDrive/Projet/Simplon/Dossier_Partager/Nutiscore/en.openfoodfacts.org.products.csv"
file_path_lea = "https://drive.google.com/file/d/1Cb7bg48nGDEN9nFrt3N07nwi6766u8Tm/view?usp=drive_link"
if os.path.exists(file_path_lea):
  file_path = file_path_lea
else:
  file_path = file_path_patoche

# **2. Exploration des données**

In [ ]:
selected_cols = [
  "code",
  "product_name",
  "quantity",
  "brands",
  "categories",
  "ingredients_text",
  "nutriscore_score",
  "nutriscore_grade",
  "energy-kj_100g",
  "energy-kcal_100g",
  "fat_100g",
  "saturated-fat_100g",
  "omega-3-fat_100g",
  "omega-6-fat_100g",
  "sugars_100g",
  "added-sugars_100g",
  "fiber_100g",
  "proteins_100g",
  "salt_100g",
  "fruits-vegetables-nuts-estimate-from-ingredients_100g",
  "nutrition-score-fr_100g"
    ]


chunksize = 10000
chunks = pd.read_csv(file_path, sep="\t", low_memory=False, header=0, chunksize=chunksize, on_bad_lines="skip")
list_df = []
for chunk in chunks:
  list_df.append(chunk[selected_cols])

# 3. **Traitement des données**

  **a. Comparaision entre nutriscore_score et nutrition-score-fr_100g **

In [ ]:
# Quels colonnes prendre entre nutriscore_score et nutrition-score-fr_100g?
df_diffnutri = None

for df in list_df:
  df_non_null_nutriscore = df[df[['nutriscore_score', 'nutrition-score-fr_100g']].notna().all(axis=1)]
  df_diffnutri = df_non_null_nutriscore[df_non_null_nutriscore['nutriscore_score'] != df_non_null_nutriscore['nutrition-score-fr_100g']]
  if len(df_diffnutri) > 0:
    break

df_diffnutri.head() # nutriscore_score car les valeurs sont communes et la colonne nommée est moins longue

df_neg_nutri = None
for df in list_df:
  df_non_null_nutriscore = df[df[['nutriscore_score', 'nutrition-score-fr_100g']].notna().all(axis=1)]
  df_neg_nutri = df_non_null_nutriscore[df_non_null_nutriscore["nutriscore_score"] < 0]
  if len(df_neg_nutri) > 0:
    break

df_neg_nutri.head() # nutriscore_score doit être gardé !

selected_cols = [
  "code",
  "product_name",
  "quantity",
  "brands",
  "categories",
  "ingredients_text",
  "nutriscore_score",
  "nutriscore_grade",
  "energy-kj_100g",
  "energy-kcal_100g",
  "fat_100g",
  "saturated-fat_100g",
  "omega-3-fat_100g",
  "omega-6-fat_100g",
  "sugars_100g",
  "added-sugars_100g",
  "fiber_100g",
  "proteins_100g",
  "salt_100g",
  "fruits-vegetables-nuts-estimate-from-ingredients_100g",
    ]


**b. Renvoyer les lignes dont le nutriscore est non vide et prendre les valeurs dont le nutri score est OK**

In [ ]:

list_df_notna = []

for df in list_df:
  if len(df) > 0:
    df_append = df[df[['nutriscore_score', 'nutriscore_grade']].notna().all(axis=1)][selected_cols]
    list_df_notna.append(df_append)

df_notna = pd.concat(list_df_notna, ignore_index=True)
df_notna.value_counts(["nutriscore_grade"])
nutri_ok = ["a", "b", "c", "d", "e"]
df_notna = df_notna[df_notna["nutriscore_grade"].isin(nutri_ok)]

print(len(df_notna))

1156038


**c. Supprimer les lignes dont toutes les données calculer vide**

In [ ]:
cols_stat = [
  "nutriscore_score",
  "nutriscore_grade",
  "energy-kj_100g",
  "energy-kcal_100g",
  "fat_100g",
  "saturated-fat_100g",
  "omega-3-fat_100g",
  "omega-6-fat_100g",
  "sugars_100g",
  "added-sugars_100g",
  "fiber_100g",
  "proteins_100g",
  "salt_100g",
  "fruits-vegetables-nuts-estimate-from-ingredients_100g",
]

df_stat_not_na = df_notna[df_notna[cols_stat].isna().all(axis=1)]

print(len(df_stat_not_na)) # il y en a aucune !

0


**d. Replacer les éléments Nan**

In [ ]:
df_notna[cols_stat] = df_notna[cols_stat].fillna(0)
df_notna.head()

,code,product_name,quantity,brands,categories,ingredients_text,nutriscore_score,nutriscore_grade,energy-kj_100g,energy-kcal_100g,fat_100g,saturated-fat_100g,omega-3-fat_100g,omega-6-fat_100g,sugars_100g,added-sugars_100g,fiber_100g,proteins_100g,salt_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g
0,1,Filet,1pcs,Body Lab,Proteinpulver,"beta alanine, creatine hcl, ancient peat & app...",12.0,d,1700.0,406.0000,6.600000,4.300000,0.0,0.0,6.70000,0.0,0.000000,80.000000,0.500000,0.013317
1,4,Almond Dark Chocolate,100g,Alter Aco,Dark chocolate bar,"Organic cocoa mass 60%, organic cane sugar, or...",19.0,e,2401.0,578.0000,40.900000,21.500000,0.0,0.0,35.00000,0.0,8.400000,7.500000,0.000000,12.000000
2,5,Bio inulin,550g,EWL,Gemüse,"Jus et purée d'abricots (50%), eau, sucre.",-2.0,a,840.0,208.0000,0.000000,0.000000,0.0,0.0,8.00000,0.0,88.000000,0.000000,0.230000,50.000000
3,6,Mac and cheese,NaN,Chef boy r dee,"Beverages and beverages preparations, Plant-ba...","Purified Water, Acidity Regulator (Citric Acid...",17.0,e,0.0,85.0000,2.400000,1.100000,0.0,0.0,2.20000,0.0,1.300000,5.700000,1.181102,0.000000
4,8,After burn,10 kg,Afterburn,Protein Powder,NaN,15.0,d,0.0,536.0715,35.714284,8.928572,0.0,0.0,28.57143,0.0,10.714286,10.714286,0.089286,0.000000


**e. Renvoyer le dataframe dans un csv ou dans un insert SQLite**

In [ ]:
# Concaténer les DataFrames en un seul DataFrame
df_clean = df_notna

# Exporter en CSV avec un séparateur de tabulation (\t)
df_clean.to_csv('openfoodfact_clean.csv', sep='\t', index=False)